# Challenge 2 Critique de Films

https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&sort=num_votes,asc&view=advanced

## Structure de l'URL

### Training

In [ ]:
# conda install -c anaconda beautifulsoup4 à faire avant dans la console
import requests
# Librairie BeautifulSoup du package bs4
from bs4 import BeautifulSoup

In [ ]:
#Solution Bruno
# Télécharger la page
url = "https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&sort=num_votes,desc"
response = requests.get(url)
content = response.content
parser = BeautifulSoup(content, 'html.parser')

print(parser.text[:5])

In [ ]:
#Solution Rod
# Télécharger la page
from requests import get
url = 'https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&sort=num_votes,desc&ref_=adv_prv'
response = requests.get(url)
print(response.text[:100])

## Structure HTML de la page

In [ ]:
from bs4 import BeautifulSoup
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

In [ ]:
movie_containers = html_soup.find_all('div', class_="lister-item mode-advanced")
print(type(movie_containers))
print(len(movie_containers))

## Extraire la data pour un seul film

In [ ]:
first_movie = movie_containers[0]
first_movie

In [ ]:
# Titre du film
first_movie.div

In [ ]:
first_movie.a

In [ ]:
first_movie.h3

In [ ]:
first_movie.h3.a

In [ ]:
# Titre du film
first_name = first_movie.h3.a.text
first_name

In [ ]:
# Année de sortie
first_year = first_movie.h3.find('span', class_="lister-item-year text-muted unbold").text
first_year

In [ ]:
# Note IMDB
first_imdb = float(first_movie.strong.text) #convertir en float (nombre à virgule)
first_imdb

In [ ]:
type(first_imdb)

In [ ]:
# Note Metacritic
first_metascore = int(first_movie.find('span', class_="metascore favorable").text) #convertir en un nombre entier
first_metascore

In [ ]:
# Nombre de votes
first_votes = first_movie.find('span', attrs = {'name' : 'nv'})
first_votes

In [ ]:
type(first_votes['data-value'])

In [ ]:
first_votes = int(first_votes['data-value'])
first_votes

## Script pour scraper une seule page

In [ ]:
test_movie_metascore = movie_containers[15].find('div', class_='ratings-metascore')
type(test_movie_metascore)

In [ ]:
# On créé des listes vides pour toutes nos informations

names = []
years = []
imdb_ratings = []
metascores = []
votes = []

# On reprend notre data movie_containers pour y extraire l'information
for container in movie_containers:
    
    #Si le film a une note metacritic on l'extrait
    if container.find('div', class_='ratings-metascore') is not None:
        #Le titre du film
        name = container.h3.a.text
        names.append(name) #on ajoute chaque élément name de la boucle à la liste names
        
        #L'année de sortie
        year = container.h3.find('span', class_='lister-item-year text-muted unbold').text
        years.append(year) #on ajoute chaque élément year de la boucle à la liste years
        
        #La note IMDB
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb) #on ajoute chaque élément imdb de la boucle à la liste imdb_ratings
        
        #Metascore
        m_score = container.find('span', class_='metascore').text
        metascores.append(int(m_score)) #on ajoute chaque élément m_score de la boucle à la liste metascores
        
        #Le nombre de votes
        vote = container.find('span', attrs = {'name' : 'nv'})['data-value']
        votes.append(int(vote)) #on ajoute chaque élément vote de la boucle à la liste votes

In [ ]:
print(len(names))

## Affichage en DataFrame avec Pandas

In [ ]:
#pip install pandas à faire d'abord dans la console
import pandas as pd
test_df = pd.DataFrame({
    'movies' : names,
    'years' : years,
    'imdb' : imdb_ratings,
    'metascores' : metascores,
    'votes' : votes
    })

In [ ]:
print(test_df.info())

In [ ]:
test_df

## Script pour toutes les pages

In [ ]:
pages = [str(i) for i in range (1,5)]
pages

In [ ]:
years_url = [str(i) for i in range (2000,2018)]
years_url

## Contrôler le taux de requêtes envoyées

In [ ]:
from time import sleep
from random import randint

for a in range(0,5):
    print('Booh')
    sleep(randint(1,4))

In [ ]:
from time import time
from IPython.display import clear_output
start_time = time() #on fixe le temps de début
requests = 0 #la variable requests va compter le nombre de requêtes
for _ in range(5):
    requests +=1 #on incrémente la variable requests d'une unité à chaque passage de boucle
    sleep(randint(1,3)) #on fait une pause de 1 à 3 secondes
    elpased_time = time() - start_time #on calcule le temps écoulé depuis la première requête
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elpased_time))
    clear_output(wait = True)

In [ ]:
from warnings import warn
warn('Attention')

## Script Final

In [ ]:
pages = [str(i) for i in range (1,500,50)]
pages

# 1. Redéclarer les listes
names = []
years = []
imdb_ratings = []
metascores = []
votes = []

# 2. Préparer l'affichage des boucles
start_time = time()
requests = 0

# 3. Pour chaque année de l'intervalle 2000-2017
for year_url in years_url:
    
    # 4. Boucle pour chaque page de 1 à 4
    for page in pages:
        
        # 5. Faire une requête GET
        response = get('https://www.imdb.com/search/title/?release_date='+ year +'-01-01,'+ year +'-12-31&sort=num_votes,desc&start='+ page)
        
        # 6. Pause de la boucle de 8 à 15 secondes
        sleep(randint(8,15))
        
        # 7. Afficher les informations sur les requêtes
        requests += 1
        elapsed_time = time() - start_time
        print('Request: {}; Frequency: {} request/s'.format(requests, requests/elapsed_time))
        clear_output(wait = True)
        
        # 8. Avertir si le code status est différent de 200
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, requests.status_code))
            
        # 9. Stopper la boucle si le nombre de requêtes est supérieur à 72
        if requests > 72:
            warn('Nombre de requêtes trop important')
            break
        
        # 10. Extraire le HTML avec BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')
            
        # 11. Sélectionner les 50 films de chaque page (container)
        mv_containers = page_html.find_all('div', class_='lister-item mode-advanced')
        
        # 12. Boucle pour chaque container
        for container in mv_containers:
            # 13. Si le film a un Metascore
            if container.find('div', class_='ratings-metascore') is not None:
                # 14. Scrape le titre
                name = container.h3.a.text
                names.append(name)
                
                # 15. Scrape l'année
                year = container.h3.find('span', class_='lister-item-year').text
                years.append(year)
                
                # 16. Scrape la note IMDB
                imdb = float(container.strong.text)
                imdb_ratings.append(imdb)
                
                # 17. Scrape le Metascore
                metascore = container.find('span', class_='metascore').text
                metascores.append(int(metascore))
                
                # 18. Scrape le nombre de votes
                vote = container.find('span', attrs = {'name':'nv'})['data-value']
                votes.append(int(vote))

In [ ]:
names